Capstone project on Data Science  
IBM Professional Certificate

0. Installing all required packages

In [2]:
#Installing folium, geocoder amd geopy
!conda install -c conda-forge geopy --yes 
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          90 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.20.0-py_0 conda-forge


geopy-1.20.0         | 57 KB     | ##################################### | 100% 
geographiclib-1.49   | 

In [3]:
!conda install -c conda-forge geocoder --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    click-7.0                  |             py_0          61 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    future-0.17.1              |        py36_1000         701 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         821 KB

The following NEW packages will be INSTALLED:

    future:   0.17.1-py36_1000 co

In [4]:
import requests
import pandas as pd
import numpy as np
import json
import matplotlib
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import folium

1. Data uploading and processing

In [6]:
# Downloading data and showing first 5 rows
#Data was scraped by the help of external service
path = 'table-1.csv'
df = pd.read_csv(path)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [7]:
# Processing of data
# First of all, we remove all rows where Borough isn't assigned
df.drop(df[df['Borough'] == 'Not assigned'].index,axis=0,inplace=True)
# If Borough is shown, but neighbourhood is unknown, we will consider that Borough = Neighbourhood
df.replace(df[df['Neighbourhood'] == 'Not assigned'].index,df['Borough'][df[df['Neighbourhood'] == 'Not assigned'].index])
# If more than one neighborhood exist in postcode area, we will combine rows
df_new = df.groupby(['Postcode'],as_index=False).agg(lambda x : x.sum() if x.dtype=='float64' else ' '.join(x))
#Remove repeating words in Borough column
newbor = [] #Create a list with clear names
for val in df_new['Borough']:
    val = val.split(' ')
    newbor.append(val[0])
df_new['Borough'] = newbor #Append it instead of old ones
print(df_new.shape)
df_new

(103, 3)


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,Rouge Malvern
1,M1C,Scarborough,Highland Creek Rouge Hill Port Union
2,M1E,Scarborough,Guildwood Morningside West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,Kingsview Village Martin Grove Gardens Richvie...
101,M9V,Etobicoke,Albion Gardens Beaumond Heights Humbergate Jam...


In [8]:
##For some reason this code doesn't work so we just download dataset
#import geocoder # import geocoder
#postal_codes = df_new['Postcode']
#lats = []
#longs = []
#for postal_code in postal_codes:
#    # initialize your variable to None
#    lat_lng_coords = None

    # loop until you get the coordinates
#    while(lat_lng_coords is None):
#        e = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#        lat_lng_coords = e.latlng
#    latitude = lat_lng_coords[0]
#    longitude = lat_lng_coords[1]
#    lats.append(latitude)
#    longs.append(longitude)

In [9]:
path = 'http://cocl.us/Geospatial_data'
geospatial = pd.read_csv(path)

In [10]:
geospatial.head()
geospatial.shape

(103, 3)

In [11]:
df_new['Latitude'] = geospatial['Latitude']
df_new['Longitude'] = geospatial['Longitude']
df_new

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,Rouge Malvern,43.806686,-79.194353
1,M1C,Scarborough,Highland Creek Rouge Hill Port Union,43.784535,-79.160497
2,M1E,Scarborough,Guildwood Morningside West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,Kingsview Village Martin Grove Gardens Richvie...,43.688905,-79.554724
101,M9V,Etobicoke,Albion Gardens Beaumond Heights Humbergate Jam...,43.739416,-79.588437


2. Visualisation of Neighbourhoods

In [12]:
# We find the coordinates of Toronto on map
address = 'Toronto'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [13]:
# We create Folium map and center it around Toronto
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)
# Then we add points corresponding to neighbourhoods
for lat, lng, borough, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['Borough'], df_new['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='red',
        fill=True,
        fill_color='#ffd800',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)  
    
toronto_map

3. Exploration of neighborhoods

In [14]:
 #Entering Foursquare creditinals
CLIENT_ID = 'your_id'
CLIENT_SECRET = 'your_secret'
VERSION = 'ver' 


In [15]:
radius = 500
LIMIT = 350
## We initiate the function to request nearby venues near with point
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
toronto_vens = getNearbyVenues(names=df_new['Neighbourhood'],
                                   latitudes=df_new['Latitude'],
                                   longitudes=df_new['Longitude']
                                  )

Rouge Malvern
Highland Creek Rouge Hill Port Union
Guildwood Morningside West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park Ionview Kennedy Park
Clairlea Golden Mile Oakridge
Cliffcrest Cliffside Scarborough Village West
Birch Cliff Cliffside West
Dorset Park Scarborough Town Centre Wexford Heights
Maryvale Wexford
Agincourt
Clarks Corners Sullivan Tam O'Shanter
Agincourt North L'Amoreaux East Milliken Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview Henry Farm Oriole
Bayview Village
Silver Hills York Mills
Newtonbrook Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park Don Mills South
Bathurst Manor Downsview North Wilson Heights
Northwood Park York University
CFB Toronto Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West Riverdale
The Beaches West Indi

In [19]:
toronto_vens.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rouge Malvern,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,Highland Creek Rouge Hill Port Union,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,Guildwood Morningside West Hill,43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,Guildwood Morningside West Hill,43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,Guildwood Morningside West Hill,43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant


In [20]:
# One Hot Enconding 
t_1hot = pd.get_dummies(toronto_vens[['Venue Category']], prefix="", prefix_sep="")
print(t_1hot.shape)
# Adding columns with heighborhoods back to data frame
t_1hot['Neighborhood'] = toronto_vens['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [t_1hot.columns[-1]] + list(t_1hot.columns[:-1])
t_1hot = t_1hot[fixed_columns]
temp = t_1hot['Yoga Studio']
t_1hot['Yoga Studio'] = t_1hot['Neighborhood']
t_1hot['Neighborhood'] = temp
t_1hot = t_1hot.rename(columns={'Yoga Studio': 'Neighborhood','Neighborhood': 'Yoga Studio'})
t_1hot.shape

(2255, 274)


(2255, 274)

In [21]:
# Next we want to see the frequency of occuring of speciic type of venue in neighborhoods
t_1hotgroup = t_1hot.groupby('Neighborhood').mean().reset_index()
t_1hotgroup

,Neighborhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Adelaide King Richmond,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03,...,0.0,0.02,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0
1,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
2,Agincourt North L'Amoreaux East Milliken Steel...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
3,Albion Gardens Beaumond Heights Humbergate Jam...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,Alderwood Long Branch,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Willowdale West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
96,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
97,Woodbine Gardens Parkview Hill,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
98,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0


In [22]:
# Next we display the frequency with showing top-5 venues
num_top_venues = 10

for neigh in t_1hotgroup['Neighborhood']:
    print(neigh)
    temp = t_1hotgroup[t_1hotgroup['Neighborhood'] == neigh].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

Adelaide King Richmond
                 venue  freq
0          Coffee Shop  0.08
1                 Café  0.05
2                  Bar  0.04
3           Steakhouse  0.04
4       Cosmetics Shop  0.03
5                Hotel  0.03
6         Burger Joint  0.03
7      Thai Restaurant  0.03
8  American Restaurant  0.03
9           Restaurant  0.03


Agincourt
                       venue  freq
0               Skating Rink   0.2
1                     Lounge   0.2
2         Chinese Restaurant   0.2
3             Sandwich Place   0.2
4             Breakfast Spot   0.2
5                Men's Store   0.0
6              Metro Station   0.0
7         Mexican Restaurant   0.0
8  Middle Eastern Restaurant   0.0
9              Movie Theater   0.0


Agincourt North L'Amoreaux East Milliken Steeles East
                             venue  freq
0                       Playground   0.5
1                             Park   0.5
2               Mexican Restaurant   0.0
3              Monument / Landmark   0.0


In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
topvens = 5
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in np.arange(topvens):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
venssort = pd.DataFrame(columns=columns)
venssort['Neighborhood'] = t_1hotgroup['Neighborhood']

for ind in np.arange(t_1hotgroup.shape[0]):
    venssort.iloc[ind, 1:] = return_most_common_venues(t_1hotgroup.iloc[ind, :], topvens)

venssort

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Adelaide King Richmond,Coffee Shop,Café,Steakhouse,Bar,Burger Joint
1,Agincourt,Lounge,Breakfast Spot,Sandwich Place,Skating Rink,Chinese Restaurant
2,Agincourt North L'Amoreaux East Milliken Steel...,Playground,Park,Women's Store,Drugstore,Dim Sum Restaurant
3,Albion Gardens Beaumond Heights Humbergate Jam...,Grocery Store,Fast Food Restaurant,Coffee Shop,Sandwich Place,Beer Store
4,Alderwood Long Branch,Pizza Place,Coffee Shop,Skating Rink,Pharmacy,Pool
...,...,...,...,...,...,...
95,Willowdale West,Butcher,Pharmacy,Pizza Place,Discount Store,Home Service
96,Woburn,Coffee Shop,Korean Restaurant,Eastern European Restaurant,Discount Store,Dog Run
97,Woodbine Gardens Parkview Hill,Fast Food Restaurant,Pizza Place,Bank,Athletics & Sports,Gastropub
98,Woodbine Heights,Park,Beer Store,Curling Ice,Cosmetics Shop,Skating Rink


4. Clustering Neighborhoods

In [24]:
#kMeans clustering code
k = 3
torontoclusters = t_1hot.groupby('Neighborhood').mean()
print(torontoclusters.shape)
print(venssort.shape)
km = KMeans(n_clusters=k, random_state=0).fit(torontoclusters)
km.labels_[0:10] #cluster labels assigned
print(km.labels_.shape)

(100, 273)
(100, 6)
(100,)


In [25]:
torontoclusters

,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
Neighborhood,,,,,,,,,,,,,,,,,,,,,
Adelaide King Richmond,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.03,0.0,...,0.0,0.02,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.0
Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
Agincourt North L'Amoreaux East Milliken Steeles East,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
Albion Gardens Beaumond Heights Humbergate Jamestown Mount Olive Silverstone South Steeles Thistletown,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
Alderwood Long Branch,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Willowdale West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
Woodbine Gardens Parkview Hill,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0


In [30]:
# Next we want to add cluster labels
#venssort.insert(0, 'Cluster Labels', km.labels_)

merged = df_new

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
merged = merged.join(venssort.set_index('Neighborhood'), on='Neighbourhood')

merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,Scarborough,Rouge Malvern,43.806686,-79.194353,2.0,Fast Food Restaurant,Dumpling Restaurant,Diner,Discount Store,Dog Run
1,M1C,Scarborough,Highland Creek Rouge Hill Port Union,43.784535,-79.160497,0.0,Bar,Women's Store,Dumpling Restaurant,Discount Store,Dog Run
2,M1E,Scarborough,Guildwood Morningside West Hill,43.763573,-79.188711,0.0,Mexican Restaurant,Intersection,Pizza Place,Breakfast Spot,Electronics Store
3,M1G,Scarborough,Woburn,43.770992,-79.216917,0.0,Coffee Shop,Korean Restaurant,Eastern European Restaurant,Discount Store,Dog Run
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,0.0,Hakka Restaurant,Bakery,Caribbean Restaurant,Athletics & Sports,Bank


In [32]:
# We find the coordinates of Toronto on map
address = 'Toronto'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude)) 
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Neighbourhood'], venssort['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=3,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The geograpical coordinate of Toronto are 43.653963, -79.387207.
